In [4]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gym
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, action_dim),
                nn.Softmax(dim=-1)
                )
        
        # critic
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, 1)
                )
        
    def forward(self):
        raise NotImplementedError
        
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device) 
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy
        
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()
    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())
                
            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())
        
# def main():
############## Hyperparameters ##############
env_name = "LunarLander-v2"
# creating environment
env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = 4
render = False
solved_reward = 230         # stop training if avg_reward > solved_reward
log_interval = 20           # print avg reward in the interval
max_episodes = 50000        # max training episodes
max_timesteps = 300         # max timesteps in one episode
n_latent_var = 64           # number of variables in hidden layer
update_timestep = 2000      # update policy every n timesteps
lr = 0.002
betas = (0.9, 0.999)
gamma = 0.99                # discount factor
K_epochs = 4                # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
random_seed = None
saved_model = True
#############################################

if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
print(lr,betas)

# logging variables
running_reward = 0
avg_length = 0
timestep = 0

# Loading a saved data to continue training or as a reference trajectory for training
#############################################
if saved_model:
#         PATH = 'PPO_LunarLander-v2-fully-trained.pth'
    PATH = 'PPO_LunarLander-v2.pth'
    ppo.policy.load_state_dict(torch.load(PATH))
    ppo.policy.eval()
#############################################


# # Loading state-action pairs
# #############################################

# # save np.load
# np_load_old = np.load

# # modify the default parameters of np.load
# # np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# # loaded_state_action_list = np.load('state_action_lunar.npy')
# loaded_state_list = np.load('state_list_lunar.npy', allow_pickle = True)
# # loaded_action_list = np.load('action_list_lunar.npy')
# # print (loaded_state_action_list)
# #############################################

#Loading Torch tensor
#############################################
# loaded_state_list = torch.load('torch_state_lunar.pt')
# print loaded_state_list
#############################################
state_count = 0 
# seen_state =[]

# training loop
for i_episode in range(1, max_episodes+1):
    state = env.reset()
    for t in range(max_timesteps):
        timestep += 1

        # Running policy_old:
        action = ppo.policy_old.act(state, memory)
        state, reward, done, _ = env.step(action)
        
#         print(loaded_state_list - state)        
#         if state in loaded_state_list:
#             state_count += 1        
#             print('State -',state)
#             print(loaded_state_list.index(state))
#             seen_state.append(state)

        # Saving reward and is_terminal:
        memory.rewards.append(reward)
        memory.is_terminals.append(done)

        # update if its time
        if timestep % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0

        running_reward += reward
        if render:
            env.render()
        if done:
            break

    avg_length += t

    # stop training if avg_reward > solved_reward
    if running_reward > (log_interval*solved_reward):
        print("########## Solved! ##########")
        torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
        break

    # logging
    if i_episode % log_interval == 0:
        avg_length = int(avg_length/log_interval)
        running_reward = int((running_reward/log_interval))

        print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
#             torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
        running_reward = 0
        avg_length = 0

print('Number of times state list was called -', state_count)
# if __name__ == '__main__':
#     main()

0.002 (0.9, 0.999)
Episode 20 	 avg length: 87 	 reward: -164
Episode 40 	 avg length: 171 	 reward: 165
Episode 60 	 avg length: 205 	 reward: 212
Episode 80 	 avg length: 226 	 reward: 216
Episode 100 	 avg length: 216 	 reward: 194
Episode 120 	 avg length: 188 	 reward: 183
Episode 140 	 avg length: 182 	 reward: 207
Episode 160 	 avg length: 214 	 reward: 179
Episode 180 	 avg length: 201 	 reward: 212
########## Solved! ##########
Number of times state list was called - 0


In [8]:
# Load and run the policy for visualization? - Thats the replacement for .eval
# Try .eval and .load with Cassie if possible?
#  Save the model using Zhaoming's code. Save the model exactly and let's see what happens?
import numpy as np

env = gym.make(env_name)
state = env.reset()
running_reward = 0
done = False
# PATH = 'PPO_LunarLander-v2.pth'
# ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
ppo.policy.load_state_dict(torch.load(PATH))
state_action_list = []
state_list = []
action_list = []
# action_list = torch.tensor()

while not done:
    state_action_list.append([state,action])
    state_list.append(state)
    
    action = ppo.policy.act(state,memory)
    env.render()
    
    state,reward,done,_ = env.step(action)
    running_reward += reward   
    
    action_list.append(torch.tensor(state))
#     print(action_list.shape)

print(running_reward)
# np.save('state_action_lunar.npy',state_action_list)
# np.save('state_list_lunar.npy',state_list)
# np.save('action_list_lunar.npy',action_list)
x = 1
torch.save(state_action_list,'saved/torch_state_action_lunar{}.pt'.format(x))
torch.save(state_list,'torch_state_lunar.pt')
torch.save(action_list,'torch_action_lunar.pt')

# loaded = (torch.load('torch_state_lunar.pt'))

# print(loaded)
# print(loaded.shape)

env.close()

265.02974287236634


In [3]:
import numpy as np
# save np.load
# np_load_old = np.load

# modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# np.load = lambda *a,**k: np_load_old(*a,allow_pickle=True)
# np.load.__defaults__=(None, True, True, 'latin1')

loaded_state_action_list = np.load('state_action_lunar.npy',allow_pickle=True)
# np.load.__defaults__=(None, False, True, 'latin1')
print (loaded_state_action_list)


[[array([ 0.005511  ,  1.426436  ,  0.27870613,  0.33193558, -0.00630981,
       -0.0624789 ,  0.        ,  0.        ], dtype=float32)
  0]
 [array([ 0.008179  ,  1.433309  ,  0.26772982,  0.30546564, -0.00722824,
       -0.01837007,  0.        ,  0.        ], dtype=float32)
  1]
 [array([ 0.01077213,  1.4395789 ,  0.25834268,  0.27866146, -0.00626514,
        0.01926388,  0.        ,  0.        ], dtype=float32)
  1]
 [array([ 0.01336536,  1.4452486 ,  0.25833926,  0.25199315, -0.00530253,
        0.01925392,  0.        ,  0.        ], dtype=float32)
  0]
 [array([ 0.0159585 ,  1.4503185 ,  0.25833625,  0.22532482, -0.00433965,
        0.01925929,  0.        ,  0.        ], dtype=float32)
  0]
 [array([ 0.01855154,  1.4547881 ,  0.25833353,  0.19865647, -0.00337692,
        0.0192563 ,  0.        ,  0.        ], dtype=float32)
  0]
 [array([ 2.1063041e-02,  1.4586663e+00,  2.4809599e-01,  1.7236911e-01,
       -3.6187694e-04,  6.0306452e-02,  0.0000000e+00,  0.0000000e+00],
      dty

In [37]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old

NameError: name 'imdb' is not defined